In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Define the bioassay function
bioassay = pd.DataFrame({
    'x': [-0.86, -0.30, -0.05, 0.73],
    'n': [5, 5, 5, 5],
    'y': [0, 1, 3, 5]
})

def bioassayfun(w, df):
    # function to be optimized
    # defines the negative log posterior = log likelihood where prior is uniform
    z = w[0] + w[1]*df['x']
    return -np.sum(df['y']*(z) - df['n']*np.log1p(np.exp(z))) 

def logl(data, a, b):
    # defines the log likelihood
    x, n, y = np.array(data['x']), np.array(data['n']), np.array(data['y'])
    a = a.reshape(-1, 1)  # Reshape a to have one column
    b = b.reshape(-1, 1)  # Reshape b to have one column
    return np.sum(y * (a + b * x) - n * np.log1p(np.exp(a + b * x)), axis=1) # returns data points in a 1D np array

# Define a function to estimate LD50
def estimate_ld50():
    # Initial guess for LD50
    ld50_guess = np.median(bioassay['x'])
    
    # Fit a logistic regression model to the data
    result = minimize(bioassayfun, [0, 1], args=(bioassay,), method='BFGS')
    a, b = result.x
    
    # Estimate the LD50 using the logistic regression model
    ld50_est = -a/b
    
    # Perform additional tests to better estimate LD50
    doses = np.linspace(-1.5, 1.5, 100)  # Define a range of dose levels
    responses = np.zeros_like(doses)  # Initialize an array to store the responses
    
    for i, dose in enumerate(doses):
        new_data = pd.DataFrame({
            'x': [dose],
            'n': [5],
            'y': [0]
        })
        
        # Compute the response at each dose level using the logistic regression model
        log_odds = a + b * new_data['x']
        prob = 1 / (1 + np.exp(-log_odds))
        responses[i] = prob
        
    # Estimate LD50 using the additional test results
    ld50_new = doses[np.abs(responses - 0.5).argmin()]
    
    return ld50_guess, ld50_est, ld50_new

# Test the function
ld50_guess, ld50_est, ld50_new = estimate_ld50()
print(f"Initial LD50 guess: {ld50_guess}")
print(f"Estimated LD50 using logistic regression: {ld50_est}")
print(f"Estimated LD50 using additional tests: {ld50_new}")


Initial LD50 guess: -0.175
Estimated LD50 using logistic regression: -0.10925288438678499
Estimated LD50 using additional tests: -0.10606060606060597
